<a href="https://colab.research.google.com/github/E-Daniel-Prada/dysgraphia-detection-crnn/blob/main/dysgraphia_detection_crnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Librerías
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, LSTM, Bidirectional, Reshape, Input, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#dimensiones de la imagen de entrada
IMG_HEIGHT, IMG_WIDTH = 128, 128
NUM_CLASSES = 7  # Número de características de disgrafía detectadas
"""
Características:
1. Escritura atípica en relación con el eje vertical, posicion inconsistente o anormal de una letra.
2. Inversiones de los grafemas. Por ejemplo cambiar b por p, entre otras.
3. El tamaño inconsistente de las letras. Este se puede comparar de forma individual o entre palabras
4. Errores ortograficos, a nivel de sintaxis. No se escribe correctamente la palabra.
5. Espaciado atipico en la escritura de palabras y frases.
6. Legibilidad pobre, es decir, no es posible determinar las letras que componen una o varias palabras
7. Palabras abandonadas, es decir, indican "murciela" en vez de "Murcielago"
"""


'\nCaracterísticas:\n1. Escritura atípica en relación con el eje vertical, posicion inconsistente o anormal de una letra.\n2. Inversiones de los grafemas. Por ejemplo cambiar b por p, entre otras.\n3. El tamaño inconsistente de las letras. Este se puede comparar de forma individual o entre palabras\n4. Errores ortograficos, a nivel de sintaxis. No se escribe correctamente la palabra.\n5. Espaciado atipico en la escritura de palabras y frases.\n6. Legibilidad pobre, es decir, no es posible determinar las letras que componen una o varias palabras\n7. Palabras abandonadas, es decir, indican "murciela" en vez de "Murcielago"\n'

In [6]:
# borrador modelo CRNN
def build_crnn():
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1))  # Imagen en escala de grises

    # Bloque de convoluciones
    x = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)

    # Aplanar y transformar para LSTM
    x = Reshape((x.shape[1] * x.shape[2], x.shape[3]))(x)  # (Transformamos para secuencias) Calcula dinámicamente el tamaño


    # Capa recurrente
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Bidirectional(LSTM(64))(x)

    # Capa completamente conectada
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [7]:
# Crear el modelo
model = build_crnn()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 128, 128, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 128, 128, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64, 64, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 32, 32, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 16, 16, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_1 (Reshape)                  │ (None, 256, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 256, 128)            │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 308,615 (1.18 MB)

 Trainable params: 308,167 (1.18 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
# Generación de datos (si las imágenes están etiquetadas)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    'ruta/dataset/',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='training')

val_generator = datagen.flow_from_directory(
    'ruta/dataset/',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='validation')


In [ ]:
# Entrenamiento
epochs = 20
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)

# Graficar resultados
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()
